<a href="https://colab.research.google.com/github/AjayBora002/Weather-Prediction/blob/main/anime_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. IMPORTING NECESSARY LIBRARIES

In [300]:
import pandas as pd
import numpy as np
import re
import nltk

In [301]:
anime = pd.read_csv('anime.csv')



In [302]:
def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9 ]', '', title.lower()).strip()

# Now apply the clean_title function

In [303]:
anime["clean_title"]=anime["name"].apply(clean_title)# this will access a new clean title column
anime["clean_synopsis"] = anime["Synopsis"].astype(str).apply(clean_title)
anime["genre"] = anime["genre"].fillna("").apply(clean_title)


COMBING TITLE , GENRE AND SYONOSIS

In [304]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine text fields
anime["combined"] = anime["clean_title"] + " " + anime["genre"] + " " + anime["clean_synopsis"]

# Vectorize
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(anime["combined"])


In [305]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def clean_input(title):
    return clean_title(title)


def search(title):
    title = clean_title(title)  # clean the input title
    query_vec = vectorizer.transform([title])  # vectorize it
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    if similarity.max() == 0:
        return anime.sample(5)  # return random if no match
    indices = np.argpartition(similarity, -5)[-5:]
    results = anime.iloc[indices].iloc[::-1]
    return results


CREATING AN INTERACTIVE BOX FOR INPUT AND OUTPUT

In [306]:
import ipywidgets as widgets
from IPython.display import display  # function used to show diiff things as output on notebook


anime_input = widgets.Text(    # creating a widget
    value='Cowboy Bebop',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)


# this whole thing will create a box to enter movie title , but wont show anything as output
  #NOW MAKING AN OUTPUT WIDGET
anime_list = widgets.Output()

def on_type(data):     # this fun will be called whenever we type something in the box
  with anime_list:
    anime_list.clear_output()

    title=data["new"]
    if len(title)>5:
      display(search(title))

anime_input.observe(on_type, names='value')  # there are diff events of the widgets whenever we input a movie name it is going to call on type and gives value event
display(anime_input, anime_list)



Text(value='Cowboy Bebop', description='Movie Title :')

Output()

In [307]:
ratings=pd.read_csv("rating.csv")
ratings.dtypes

,0
user_id,int64
anime_id,int64
rating,int64


FINDING SIMILAR USERS ON THE BASIS OF RATINGS

In [308]:
animeid=1

In [309]:
similar_users= ratings[(ratings["anime_id"] == animeid) & (ratings["rating"] > 4)]["user_id"].unique()

similar_user_recs = ratings[(ratings["user_id"].isin(similar_users)) & (ratings["rating"] > 4)]["anime_id"]
# this will show the movies that are rated by similar users and find their used ids

In [310]:
similar_user_recs=similar_user_recs.value_counts()/len(similar_users) # this counts the no of 5 rates is given to a movie and then div by len shows recommending percent like top one has 100%

# we are doing this to find per of similar people who watched the movie
similar_user_recs = similar_user_recs[similar_user_recs > .10]  # shows movies having per greater than 10 %

In [311]:
all_users=ratings[(ratings)["anime_id"].isin(similar_user_recs.index) & (ratings["rating"]>4)]
all_users_recs=all_users["anime_id"].value_counts()/len(all_users["user_id"].unique())# this will give us percentage of all the users who liked the movies

In [312]:
rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)# this concatenates the data of how much similar people to us liked and how much avg person liked it
rec_percentages.columns=["similar","all"]

In [313]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
rec_percentages=rec_percentages.sort_values("score",ascending=False)   # using pandas sort method to make an order of the score the higher the score more good will be the recommendation
rec_percentages

,similar,all,score
anime_id,,,
1,1.000000,0.196142,5.098357
5,0.389843,0.084404,4.618782
400,0.152502,0.038481,3.963017
570,0.103137,0.028154,3.663284
1566,0.103809,0.028389,3.656716
...,...,...,...
14289,0.112173,0.103974,1.078858
3457,0.176923,0.167138,1.058546
7054,0.190366,0.185155,1.028142


In [314]:
rec_percentages.head(10).merge(anime,left_index=True,right_on="anime_id")

,similar,all,score,anime_id,name,genre,Synopsis,Type,Producer,Studio,...,ScoredBy,Popularity,Members,episodes,Source,Aired,Link,clean_title,clean_synopsis,combined
0,1.000000,0.196142,5.098357,1,Cowboy Bebop,action adventure comedy drama scifi space,"In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],...,363889.0,39.0,704490,26.0,Original,"Apr 3, 1998 to Apr 24, 1999",https://myanimelist.net/anime/1/Cowboy_Bebop,cowboy bebop,in the year 2071 humanity has colonized severa...,cowboy bebop action adventure comedy drama sci...
1,0.389843,0.084404,4.618782,5,Cowboy Bebop: Tengoku no Tobira,action space drama mystery scifi,"Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],...,111187.0,475.0,179899,1.0,Original,"Sep 1, 2001",https://myanimelist.net/anime/5/Cowboy_Bebop__...,cowboy bebop tengoku no tobira,another day another bountysuch is the life of ...,cowboy bebop tengoku no tobira action space dr...
15087,0.152502,0.038481,3.963017,400,Seihou Bukyou Outlaw Star,,Gene Starwind has always dreamed of piloting h...,TV,NaN,NaN,...,NaN,1016.0,109861,NaN,NaN,"Jan 9, 1998 to Jun 26, 1998",https://myanimelist.net/anime/400/Seihou_Bukyo...,seihou bukyou outlaw star,gene starwind has always dreamed of piloting h...,seihou bukyou outlaw star gene starwind has a...
482,0.103137,0.028154,3.663284,570,Jin-Rou,military police psychological drama romance,After witnessing the suicide bombing of a terr...,Movie,NaN,['Production I.G'],...,40561.0,1061.0,84815,1.0,Manga,"Jun 3, 2000",https://myanimelist.net/anime/570/Jin-Rou,jinrou,after witnessing the suicide bombing of a terr...,jinrou military police psychological drama rom...
1284,0.103809,0.028389,3.656716,1566,Koukaku Kidoutai: Stand Alone Complex - Solid ...,mecha military mystery police scifi seinen,A.D. 2034. It has been two years since Motoko ...,Special,NaN,['Production I.G'],...,34251.0,1374.0,69085,1.0,Manga,"Sep 1, 2006",https://myanimelist.net/anime/1566/Koukaku_Kid...,koukaku kidoutai stand alone complex solid st...,ad 2034 it has been two years since motoko kus...,koukaku kidoutai stand alone complex solid st...
653,0.188424,0.051606,3.651193,801,Koukaku Kidoutai: Stand Alone Complex 2nd GIG,action military scifi mystery police mecha seinen,"Following the closure of the ""Laughing Man"" ca...",TV,"['Bandai Visual', 'Dentsu', 'Victor Entertainm...",['Production I.G'],...,72115.0,626.0,149102,26.0,Manga,"Jan 1, 2004 to Jan 8, 2005",https://myanimelist.net/anime/801/Koukaku_Kido...,koukaku kidoutai stand alone complex 2nd gig,following the closure of the laughing man case...,koukaku kidoutai stand alone complex 2nd gig a...
301,0.109858,0.030176,3.640623,329,Planetes,drama romance scifi seinen space,"In 2075, space travel is no longer just a drea...",TV,"['Bandai Visual', 'NHK']",['Sunrise'],...,46131.0,656.0,138203,26.0,Manga,"Oct 4, 2003 to Apr 17, 2004",https://myanimelist.net/anime/329/Planetes,planetes,in 2075 space travel is no longer just a dream...,planetes drama romance scifi seinen space in 2...
3226,0.133906,0.038013,3.522680,4106,Trigun: Badlands Rumble,action scifi adventure comedy drama shounen,20 years after meddling into the bank heist o...,Movie,"['flying DOG', 'Sunny Side Up', 'Studio Jack',...",['Madhouse'],...,46942.0,1117.0,84011,1.0,Manga,"Apr 2, 2010",https://myanimelist.net/anime/4106/Trigun__Bad...,trigun badlands rumble,20 years after meddling into the bank heist of...,trigun badlands rumble action scifi adventure ...
406,0.271695,0.077812,3.491683,467,Koukaku Kidoutai: Stand Alone Complex,action military scifi police mecha seinen,"In the not so distant future, mankind has adva...",TV,"['Bandai Visual', 'Dentsu', 'Victor Entertainm...",['Production I.G'],...,110151.0,331.0,251675,26.0,Manga,"Oct 1, 2002 to Oct 1, 2003",https://myanimelist.net/anime/467/Koukaku_Kido...,koukaku kidoutai stand alone complex,in the not so distant future mankind has advan...,koukaku kidoutai stand alone

In [315]:
def find_similar_anime(animeid):

    similar_users = ratings[(ratings["anime_id"] == animeid)
    & (ratings["rating"] > 4)]["user_id"].unique()
    similar_user_recs = ratings[(ratings["user_id"].isin(similar_users)) # this is finding recommendation similar to us
    & (ratings["rating"] > 4)]["anime_id"]

    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)# adjusting the per of recommendation
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    all_users = ratings[(ratings["anime_id"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]# this is finding common recommendation among all of the users
    all_user_recs = all_users["anime_id"].value_counts() / len(all_users["user_id"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)# concatinating
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]# generating score

    rec_percentages=rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(anime, left_index=True, right_on="anime_id")[["score", "name", "genre","episodes"]]  #sorting and returning our merge data

In [316]:
anime_name_input = widgets.Text(    # creating a widget
    value='Cowboy Bebop',
    description = "Movie Title :",  # anime title will be entered here
    disabled=False   # it will be enabled
)

recommendation_list=widgets.Output()

def on_type(data):
  with recommendation_list:
    recommendation_list.clear_output()# removes old output
    title=data["new"]
    if len(title)>5:
      results=search(title)
      animeid=results.iloc[0]["anime_id"]
      display(find_similar_anime(animeid))

anime_name_input.observe(on_type,names="value")
display(anime_name_input, recommendation_list)

Text(value='Cowboy Bebop', description='Movie Title :')

Output()